In [1]:
from csg.core import CSG
from csg.geom import Vertex, Vector, Plane, Polygon
import meshplot as mp
import numpy as np
import geo_vase

def plotCsg(csg):
    verts, polys, count = csg.toVerticesAndPolygons()
    triangles = []

    tris = []
    # polys are always convex (apparently...)
    for poly in polys:
        vlen = len(poly)
        for n in range(1,vlen-1):
            tris.append(
                [
                    poly[0],
                    poly[n%vlen],
                    poly[(n+1)%vlen],
                ]
            )
    mp.plot(np.array(verts), np.array(tris), shading={"point_size": 0.1})
bez_nodes = [
        [0.1,  0.5, 0.5], 
        [0.00, 0.0, 0.25], 
        [0.00, 0.00, 0.00] # needs 0s to be 3D :)
    ]
gv = geo_vase.GeoVase(
    rows=7,
    sides=7,
    spiral=0,
    bez_nodes=bez_nodes
)

# Nice --- easy to convert from our mesh to CSG mesh.
ps = CSG.fromPolygons([Polygon([Vertex(gv.verts[v].tolist()) for v in reversed(f)]) for f in gv.faces])
b = CSG.fromPolygons([Polygon([Vertex(gv.verts[v].tolist()) for v in reversed(f)]) for f in gv.faces])
b.rotate([0, 1, 0], 30)

plotCsg(ps)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0055555…

In [11]:
def frameCube(center, frame, scale):
    n, u, v = frame
    verts = []
    
    for i in range(8):
        a = (2 * bool(i & 1) - 1)
        b = (2 * bool(i & 2) - 1)
        c = (2 * bool(i & 4) - 1)

        verts.append(Vertex((center - (n*a + u*b + v*c)*scale).tolist()))
    polys = []
    cubeFaces = [
        [[0, 4, 6, 2], [-1, 0, 0]],
        [[1, 3, 7, 5], [+1, 0, 0]],
        [[0, 1, 5, 4], [0, -1, 0]],
        [[2, 6, 7, 3], [0, +1, 0]],
        [[0, 2, 3, 1], [0, 0, -1]],
        [[4, 5, 7, 6], [0, 0, +1]]
    ]
    
    for f,_ in cubeFaces:
        polys.append(Polygon([ verts[v] for v in f]))

    return CSG.fromPolygons(polys)

bez_nodes = [
        [0.1,  0.4, 0.5], 
        [0.00, 0.0, 0.5], 
        [0.00, 0.00, 0.00] # needs 0s to be 3D :)
    ]

gv = geo_vase.GeoVase(
    rows=10,
    sides=9,
    spiral=0,
    bez_nodes=bez_nodes,
    weights=[0, 0.5, 0.8, 0.8],

)

c = CSG.cube(center=[0,0,0], radius=[10, 0.4, 10])

for i, vert in enumerate(gv.verts):
    vnorms = gv.vnorms[i]
    n, u, v = vnorms
    r = 0.05
    q = 10
    c = c - frameCube(vert - n*q, vnorms, q + r)


plotCsg(c)
verts, polys, count = c.toVerticesAndPolygons()
# print({
#     "vertices": verts,
#     "faces": [[x for x in reversed(p)] for p in polys]
# })

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003528…